<a href="https://colab.research.google.com/github/reveondivad/ExData_Plotting1/blob/master/distilGPT2_finetuned_MA_Meditations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tune distilGPT-2 on Marco Aurelio Medidations for text generation

> Author: [Manuel Romero / mrm8488](https://twitter.com/mrm8488)

## Get the data

In [1]:
!wget https://raw.githubusercontent.com/GITenberg/Meditations_2680/master/2680.txt -O text.txt

--2020-11-03 21:32:51--  https://raw.githubusercontent.com/GITenberg/Meditations_2680/master/2680.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 425173 (415K) [text/plain]
Saving to: ‘text.txt’

text.txt            100%[===================>] 415.21K  --.-KB/s    in 0.05s   

2020-11-03 21:32:51 (8.65 MB/s) - ‘text.txt’ saved [425173/425173]



### Remove book index, introduction an so on

In [2]:
!wc -l text.txt

7212 text.txt


In [3]:
!head -88 text.txt

The Project Gutenberg EBook of Meditations, by Marcus Aurelius

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Meditations

Author: Marcus Aurelius

Posting Date: December 25, 2008 [EBook #2680]
Release Date: June, 2001

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK MEDITATIONS ***




Produced by J. Boulton





MEDITATIONS

By Marcus Aurelius




CONTENTS


     NOTES

     INTRODUCTION

     FIRST BOOK

     SECOND BOOK

     THIRD BOOK

     FOURTH BOOK

     FIFTH BOOK

     SIXTH BOOK

     SEVENTH BOOK

     EIGHTH BOOK

     NINTH BOOK

     TENTH BOOK

     ELEVENTH BOOK

     TWELFTH BOOK

     APPENDIX

     GLOSSARY




Original Transcriber's Notes:

This text was scanned by J. Boulton using Textbridge OCR. The Greek
port

In [4]:
!cat text.txt | tail -7124 >> train.txt

In [5]:
!head -20 train.txt

INTRODUCTION


MARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name
was M. Annius Verus, and he was sprung of a noble family which claimed
descent from Numa, second King of Rome. Thus the most religious of
emperors came of the blood of the most pious of early kings. His father,
Annius Verus, had held high office in Rome, and his grandfather, of
the same name, had been thrice Consul. Both his parents died young, but
Marcus held them in loving remembrance. On his father's death Marcus
was adopted by his grandfather, the consular Annius Verus, and there was
deep love between these two. On the very first page of his book Marcus
gratefully declares how of his grandfather he had learned to be gentle
and meek, and to refrain from all anger and passion. The Emperor Hadrian
divined the fine character of the lad, whom he used to call not Verus
but Verissimus, more Truthful than his own name. He advanced Marcus to
equestrian rank when six years of age, and at the age of eight m

In [6]:
!rm -rf text.txt

## Install HF Transformers

In [7]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 49361 (delta 15), reused 30 (delta 4), pack-reused 49311
Receiving objects: 100% (49361/49361), 36.51 MiB | 18.12 MiB/s, done.
Resolving deltas: 100% (34423/34423), done.


In [8]:
!pip install -q ./transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 3.5MB/s 
     |████████████████████████████████| 2.9MB 17.1MB/s 
     |████████████████████████████████| 1.1MB 27.3MB/s 


## Fine tuning the model

In [9]:
!ls transformers/examples/language-modeling/
!pwd
!pip install datasets

README.md  run_clm.py  run_mlm.py  run_mlm_wwm.py  run_plm.py
/content
     |████████████████████████████████| 153kB 3.4MB/s 
     |████████████████████████████████| 17.7MB 1.2MB/s 
     |████████████████████████████████| 245kB 58.0MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [10]:
##!python /content/transformers/examples/language-modeling/run_clm.py \
#--model_type=gpt2 \
#--model_name_or_path=distilgpt2 \
#--do_train \
#--train_data_file=/content/train.txt \
#--num_train_epochs 100 \
#--output_dir model_output \
#--overwrite_output_dir \
#--save_steps 20000 \
#--per_gpu_train_batch_size 4
!mkdir /content/tmp
!python /content/transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=distilgpt2 \
    --train_file=/content/train.txt \
    --do_train \
    --output_dir=/content/tmp
    

2020-11-03 21:33:24.853111: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/03/2020 21:33:26 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
11/03/2020 21:33:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/tmp', overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov03_21-33-26_c769d5f80795', logging_first_ste

## Generate text with our fine tuned model

In [13]:
!python /content/transformers/examples/text-generation/run_generation.py \
--model_type=gpt2 \
--model_name_or_path=/content/tmp \
--num_return_sequences 5 \
--length 100 \


2020-11-03 21:40:38.539734: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/03/2020 21:40:40 - WARNING - __main__ -   device: cpu, n_gpu: 0, 16-bits training: False
11/03/2020 21:40:44 - INFO - __main__ -   Namespace(device=device(type='cpu'), fp16=False, k=0, length=100, model_name_or_path='/content/tmp', model_type='gpt2', n_gpu=0, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prefix='', prompt='', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Model prompt >>> 
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
<|endoftext|>"…What are you, Mad Men?"


When Beni, 25, writes to him in his memoirs on Christmas Day, having died, and disinterred and attempted to suppress his thought, it was said that he had used a second of him as an antidote. (The line is that this is true, and, forgive us, it is true, to us, ind